<a href="https://colab.research.google.com/github/kundana22bce8985/knowledge-based-search-engine/blob/main/your_notebook_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu PyPDF2 transformers torch


   ââââââââââââââââââââââââââââââââââââââââ 31.4/31.4 MB 41.7 MB/s eta 0:00:00
   ââââââââââââââââââââââââââââââââââââââââ 232.6/232.6 kB 13.6 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2
import PyPDF2

file_path = "/content/Resume.pdf"

# Open PDF file
with open(file_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

print(text[:500])  # print first 500 characters


K. KUNDANA
kundanasri2004@gmail.com âLinkedIn âGitHub
OBJECTIVE
Being ambitious and hardworking, I am seeking a position in a company where I can utilize my knowledge and build
valuable skills. I am committed to not only applying my existing knowledge but also to actively developing new skills
that will contribute to both my personal and professional growth.
EDUCATION
Bachelor of Computer Science , Vellore Institute of Technology Expected 2026
CGPA: 8.27
Intermediate , Shastra Junior College 202


In [ ]:
import os
from PyPDF2 import PdfReader

doc_folder = './'  # Change to your folder if using Drive
documents = []

for file in os.listdir(doc_folder):
    if file.endswith(".pdf"):
        reader = PdfReader(os.path.join(doc_folder, file))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + " "
        documents.append(text)

print(f"Loaded {len(documents)} documents")


Loaded 1 documents


In [ ]:
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

all_chunks = []
for doc in documents:
    all_chunks.extend(chunk_text(doc))

print(f"Total chunks: {len(all_chunks)}")


Total chunks: 1


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(all_chunks, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print(f"FAISS index has {index.ntotal} vectors")


FAISS index has 1 vectors


In [ ]:
def search(query, k=5):
    query_vec = model.encode([query])
    distances, indices = index.search(np.array(query_vec), k)
    return [all_chunks[i] for i in indices[0]]

# Example:
query = "What are the symptoms of plant disease X?"
results = search(query)
for i, chunk in enumerate(results):
    print(f"Chunk {i+1}: {chunk[:200]}...")  # Show first 200 chars


Chunk 1: K. KUNDANA kundanasri2004@gmail.com âLinkedIn âGitHub OBJECTIVE Being ambitious and hardworking, I am seeking a position in a company where I can utilize my knowledge and build valuable skills. I am c...
Chunk 2: K. KUNDANA kundanasri2004@gmail.com âLinkedIn âGitHub OBJECTIVE Being ambitious and hardworking, I am seeking a position in a company where I can utilize my knowledge and build valuable skills. I am c...
Chunk 3: K. KUNDANA kundanasri2004@gmail.com âLinkedIn âGitHub OBJECTIVE Being ambitious and hardworking, I am seeking a position in a company where I can utilize my knowledge and build valuable skills. I am c...
Chunk 4: K. KUNDANA kundanasri2004@gmail.com âLinkedIn âGitHub OBJECTIVE Being ambitious and hardworking, I am seeking a position in a company where I can utilize my knowledge and build valuable skills. I am c...
Chunk 5: K. KUNDANA kundanasri2004@gmail.com âLinkedIn âGitHub OBJECTIVE Being ambitious and hardworking, I am seeking a positio

In [ ]:
import ipywidgets as widgets
from IPython.display import display

query_box = widgets.Text(
    value='',
    placeholder='Ask a question...',
    description='Query:',
    layout=widgets.Layout(width='100%')
)

display(query_box)


Text(value='', description='Query:', layout=Layout(width='100%'), placeholder='Ask a question...')

ð Query: W
ð§  Answer: Eclipse Projects Plant Disease
ð Query: What are pr
ð§  Answer: Java
ð Query: What are projects?
ð§  Answer: Eclipse Projects Plant Disease Detection


In [ ]:
from transformers import pipeline

qa_model = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [ ]:
# ð§  Function to generate an answer
def generate_answer(query):
    context = " ".join(search(query))
    # Pass the question and context separately to the QA model
    answer = qa_model(question=query, context=context, max_length=200)['answer']
    return answer

# ð§  Function to handle query input and generate answer
def on_submit(change):
    query = change['new']
    if query:
        print("ð Query:", query)
        answer = generate_answer(query)
        print("ð§  Answer:", answer)

# ð Connect the query box to the handler
query_box.observe(on_submit, names='value')